In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [3]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [4]:
environment = jinja2.Environment()

In [5]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [6]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [7]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [8]:
typespec

{'reasoning': 'For a gym tracking bot, I expect users to send messages like:\n- "I did 3 sets of bench press with 80kg"\n- "Show my progress for bench press"\n- "Suggest a 30-minute workout for chest with dumbbells"\n\nThe LLM needs to extract:\n1. Exercise details (name, sets, reps, weight)\n2. Time periods for progress tracking\n3. Equipment availability and time constraints for suggestions\n\nKey operations:\n- Record completed exercises\n- Track progress over time\n- Generate workout plans based on constraints\n- List available equipment (to know what can be used in routines)\n\nThe Exercise model needs to capture:\n- Exercise name\n- Sets, reps, weight used\n- Equipment required\n- Target muscle groups',
 'typespec_definitions': 'model Exercise {\n    name: string;\n    sets: integer;\n    reps: integer;\n    weight: float;\n    equipment: string;\n    muscleGroups: string[];\n}\n\nmodel WorkoutPlan {\n    duration: duration;\n    exercises: Exercise[];\n    restBetweenSets: durat

In [9]:
print(typespec["reasoning"])

For a gym tracking bot, I expect users to send messages like:
- "I did 3 sets of bench press with 80kg"
- "Show my progress for bench press"
- "Suggest a 30-minute workout for chest with dumbbells"

The LLM needs to extract:
1. Exercise details (name, sets, reps, weight)
2. Time periods for progress tracking
3. Equipment availability and time constraints for suggestions

Key operations:
- Record completed exercises
- Track progress over time
- Generate workout plans based on constraints
- List available equipment (to know what can be used in routines)

The Exercise model needs to capture:
- Exercise name
- Sets, reps, weight used
- Equipment required
- Target muscle groups


In [10]:
print(typespec["typespec_definitions"])

model Exercise {
    name: string;
    sets: integer;
    reps: integer;
    weight: float;
    equipment: string;
    muscleGroups: string[];
}

model WorkoutPlan {
    duration: duration;
    exercises: Exercise[];
    restBetweenSets: duration;
}

model Equipment {
    name: string;
    type: string;
    isAvailable: boolean;
}

interface GymBot {
    @llm_func(2)
    recordExercise(exercise: Exercise): void;

    @llm_func(1)
    trackProgress(exerciseName: string, from: utcDateTime, to: utcDateTime): Exercise[];

    @llm_func(1)
    suggestWorkout(duration: duration, targetMuscles: string[], availableEquipment: string[]): WorkoutPlan;

    @llm_func(1)
    updateEquipment(equipment: Equipment[]): void;
}


In [11]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [12]:
print(drizzle["drizzle_schema"])

import { integer, pgTable, text, real, boolean, timestamp, array } from "drizzle-orm/pg-core";
import { relations } from "drizzle-orm";

// Muscle groups table
export const muscleGroupsTable = pgTable("muscle_groups", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull().unique()
});

// Equipment table
export const equipmentTable = pgTable("equipment", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull(),
  type: text("type").notNull(),
  isAvailable: boolean("is_available").notNull().default(true)
});

// Exercises table
export const exercisesTable = pgTable("exercises", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull(),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight").notNull(),
  equipmentId: integer("equipment_id").references(() => equipmentTable.id)
});

// Exercise to muscle groups junction table
export cons

In [13]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [14]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups.',
  'examples': ['I did 3 sets of 12 bench presses with 135 pounds',
   'Just finished 4 sets of squats at 225 lbs',
   'I want to log my deadlift session',
   'Record my exercise routine for today',
   'Add my workout: 3 sets of pull-ups']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical data for a specific exercise over a given time period to track progress and improvements',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'Track my deadlift numbers from last week',
   "What's my progression on pull-ups?",
   'Show exercise history for bicep curls']},
 {'name': 'suggestWorkout',
  'description': 'Generates a personalized workout plan based on available time, target muscle groups, and available equipment',
  'examples': ['

In [15]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [16]:
print(pre_response.content[0].text)

<instructions>
The updateEquipment function expects an array of Equipment objects. Each Equipment object must have:
- name: string (descriptive name of the equipment)
- type: string (category/type of equipment)
- isAvailable: boolean (availability status)

When processing user input:
1. Look for equipment names, types, and availability status
2. Convert natural language descriptions into structured Equipment objects
3. Package multiple equipment updates into an array
4. Default isAvailable to true if not explicitly specified
</instructions>

<examples>
    <example>
        <input>Barbell and dumbbells are now available in the weight room</input>
        <output>[
    {
        "name": "Barbell",
        "type": "Free Weights",
        "isAvailable": true
    },
    {
        "name": "Dumbbells",
        "type": "Free Weights",
        "isAvailable": true
    }
]</output>
    </example>

    <example>
        <input>Leg press machine is broken and under maintenance</input>
        <out

In [17]:
pre_processors

{'recordExercise': {'instructions': 'The recordExercise function expects a complete Exercise object with the following rules:\n1. name: Must be a valid exercise name in lowercase\n2. sets: Must be a positive integer between 1-10\n3. reps: Must be a positive integer between 1-100\n4. weight: Must be a positive float number (in kg/lbs)\n5. equipment: Must specify the equipment used\n6. muscleGroups: Must include at least one target muscle group\n\nIf user input is incomplete, infer reasonable defaults based on standard exercise patterns.',
  'examples': [('bench press 80x6',
    '{\n    "name": "bench press",\n    "sets": 3,\n    "reps": 6,\n    "weight": 80.0,\n    "equipment": "barbell",\n    "muscleGroups": ["chest", "triceps", "shoulders"]\n}'),
   ('did 4 sets of 12 pushups',
    '{\n    "name": "pushup",\n    "sets": 4,\n    "reps": 12,\n    "weight": 0.0,\n    "equipment": "bodyweight",\n    "muscleGroups": ["chest", "triceps", "shoulders"]\n}'),
   ('lat pulldown 3x10 50kg',
    

In [18]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)

In [19]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { \n    exercisesTable, \n    equipmentTable, \n    muscleGroupsTable, \n    exerciseMuscleGroupsTable,\n    exerciseHistoryTable \n} from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (exercise: Exercise): Promise<void> => {\n    // First, get or create equipment\n    const [equipmentRecord] = await db\n        .select()\n        .from(equipmentTable)\n        .where(eq(equipmentTable.name, exercise.equipment))\n        .limit(1);\n\n    let equipmentId: number;\n    if (!equipmentRecord) {\n        const [newEquipment] = await db\n            .insert(equipmentTable)\n            .values({ name: exercise.equipment, type: \'unknown\', isAvailable: true })\n            .returning({ id: equipmentTable.id });\n        equipmentId = newEquipment.id;\n    } else {\n        equipmentId = equipmentRecord.id;\n    }\n\n    // Insert the exercise\n    const [exerciseRecord] = await d

In [20]:
with open("core/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [21]:
hdl_key = "recordExercise"
params = {
    "handler": handlers[hdl_key]["handler"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

In [22]:
print(file_content)

import { Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { 
    exercisesTable, 
    equipmentTable, 
    muscleGroupsTable, 
    exerciseMuscleGroupsTable,
    exerciseHistoryTable 
} from '../db/schema/application';
import { eq } from 'drizzle-orm';

const handle = async (exercise: Exercise): Promise<void> => {
    // First, get or create equipment
    const [equipmentRecord] = await db
        .select()
        .from(equipmentTable)
        .where(eq(equipmentTable.name, exercise.equipment))
        .limit(1);

    let equipmentId: number;
    if (!equipmentRecord) {
        const [newEquipment] = await db
            .insert(equipmentTable)
            .values({ name: exercise.equipment, type: 'unknown', isAvailable: true })
            .returning({ id: equipmentTable.id });
        equipmentId = newEquipment.id;
    } else {
        equipmentId = equipmentRecord.id;
    }

    // Insert the exercise
    const [exercise

In [23]:
print(handlers["suggestWorkout"]["handler"])

import { db } from "../db";
import { 
    exercisesTable,
    equipmentTable,
    exerciseMuscleGroupsTable,
    muscleGroupsTable,
    workoutPlansTable,
    workoutPlanExercisesTable
} from '../db/schema/application';
import { eq, inArray, and } from 'drizzle-orm';

const handle = async (
    duration: number,
    targetMuscles: string[],
    availableEquipment: string[]
): Promise<{
    duration: number,
    exercises: Array<{
        name: string,
        sets: number,
        reps: number,
        weight: number,
        equipment: string,
        muscleGroups: string[]
    }>,
    restBetweenSets: number
}> => {
    // Get muscle group IDs for target muscles
    const muscleGroups = await db
        .select({ id: muscleGroupsTable.id })
        .from(muscleGroupsTable)
        .where(inArray(muscleGroupsTable.name, targetMuscles));

    const muscleGroupIds = muscleGroups.map(mg => mg.id);

    // Get exercises that match the muscle groups and available equipment
    const exerci

In [24]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [25]:
ts_defs

{'reasoning': 'I expect user to send messages like "I ate a burger" or "I had a salad for lunch".\nLLM can extract and infer the arguments from plain text and pass them to the handler\n"I ate a burger" -> recordDish({name: "burger", ingredients: [\n    {name: "bun", calories: 200},\n    {name: "patty", calories: 300},\n    {name: "lettuce", calories: 10},\n    {name: "tomato", calories: 20},\n    {name: "cheese", calories: 50},\n]})\n- recordDish(dish: Dish): void;\n...',
 'typespec_definitions': 'model Dish {\n    name: string;\n    ingredients: Ingredient[];\n}\n\nmodel Ingredient {\n    name: string;\n    calories: integer;\n}\n\ninterface DietBot {\n    @llm_func(1)\n    recordDish(dish: Dish): void;\n    @llm_func(1)\n    listDishes(from: utcDateTime, to: utcDateTime): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [26]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [27]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [28]:
from shutil import copytree, ignore_patterns

In [29]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [30]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [31]:
from core.stages import router

In [32]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [33]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups. Use this when the user wants to log or record their exercise activity.',
  'examples': ['I want to record my exercise routine for today',
   'I just did 3 sets of bench press',
   'Log my workout: 4 sets of squats with 200lbs',
   'Add my deadlift session to my log']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specific exercise over a given time period. Use this when the user wants to see their progress or history for a particular exercise.',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'What were my deadlift numbers from last week?',
   'Track my progress on pull-ups']},
 {'name': 'suggestWorkout',
  'description': 'Generates a workout plan based on desired duration, target muscle groups, and av

In [47]:
from core.stages import typespec
from core.stages import tsp_compiler

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

typespec_errors = ""
attempts_left = 3

while attempts_left > 0:
    attempts_left -= 1
    
    prompt = tsp_tpl.render(
        application_description=application_description,
        typespec_errors=typespec_errors,
    )
  
    if attempts_left >= 2:
        updated_prompt = prompt + "// PS. Make sure to eject a random error in TypeSpec output for testing purposes"
    else:
        updated_prompt = prompt
  
    tsp_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": updated_prompt}]
    )
    
    typespec = stages.typespec.parse_output(tsp_response.content[0].text)

    # Write TypeSpec definitions to file
    with open('./test/tsp_schema/notebook.tsp', 'w') as f:
        f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        f.write(typespec["typespec_definitions"])

    compiler = tsp_compiler.TypeSpecCompiler('./test/tsp_schema/')
    result = compiler.compile('./notebook.tsp')
    
    print(result)
    
    if result["result"] == tsp_compiler.TypeSpecCompilationStatus.COMPILATION_ERROR:
        typespec_errors = result["errors"]
        print(typespec_errors)
        continue
    else:
        break


In [90]:
typespec


{'reasoning': 'For a finance tracking app, users would likely send messages like:\n- "I spent $50 on groceries yesterday"\n- "Got my salary of $3000 today"\n- "Show me my expenses for last month"\n\nThe LLM would need to extract:\n1. Transaction type (expense/income)\n2. Amount\n3. Category\n4. Date\n5. Optional description/notes\n\nFor reports, users might ask for:\n- Time-based summaries\n- Category-based breakdowns\n- Balance calculations\n\nThe interface should support recording transactions and generating reports while keeping the argument structure simple enough to be extracted from natural language.',
 'typespec_definitions': 'model Transaction {\n  type: string; // "expense" or "income"\n  amount: decimal;\n  category: string;\n  date: utcDateTime;\n  description: string;\n}\n\nmodel TransactionSummary {\n  totalIncome: decimal;\n  totalExpenses: decimal;\n  balance: decimal;\n  transactions: Transaction[];\n}\n\ninterface FinanceTracker {\n  @llm_func(1)\n  recordTransaction(t

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
